# New sequence families from unclassified structural regions

We ask the question: what is the coverage of sequence families in a structural dataset? We hope to identify unclassified structural regions that are potentially new sequence families.

We will use Pfam family definitions as the sequence classification data and the structures in the PDB.

Authors: 
- Kyle Stiers (kylestiers [at] gmail.com)
- Aleix Lafita (aleixlafita [at] gmail.com)

In [1]:
from pyspark.sql import Row,SparkSession
from pyspark import SparkContext

from mmtfPyspark.structureViewer import view_binding_site
from mmtfPyspark.ml import pythonRDDToDataset
from mmtfPyspark.io import mmtfReader
from mmtfPyspark.filters import PolymerComposition
from mmtfPyspark.mappers import StructureToPolymerChains
from mmtfPyspark.structureViewer import view_structure

import pandas as pd
import matplotlib
import py3Dmol

/Users/peter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/peter/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


## Configure Spark

In [2]:
spark = SparkSession.builder.master("local[4]").appName("sequence_structure_coverage").getOrCreate()
sc = spark.sparkContext

## Parse structural information in the PDB

In [ ]:
# Set the location of the PDB sample to analyze
path = "../resources/mmtf_reduced_sample"
pdb = mmtfReader.read_sequence_file(path, fraction=0.1)

pdb.count()

## Load Pfam data

Data file downloaded from the Pfam FTP (ftp://ftp.ebi.ac.uk/pub/databases/Pfam/current_release/database_files/).
It contains a mapping of the regions in PDB structures classified by Pfam.

In [ ]:
pfam_data = pd.read_csv("pdb_pfamA_reg.txt",sep='\t',header=(0), low_memory=False)
pfam_data.head(5)

### Trim dataset to only keep information we need

In [ ]:
d = {'chainId': pfam_data['pdb_id']+'.'+pfam_data['chain'], 'pfam':pfam_data['pfamA_acc'], 'pdb_res_start': pfam_data['pdb_res_start'], 'pdb_res_end':pfam_data['pdb_res_end'], 'range':pfam_data['pdb_res_end']-pfam_data['pdb_res_start']}
df = pd.DataFrame(data=d)
df_noDups = df.drop_duplicates(subset="chainId")
df_noDups.head(5)

### Flatmap PDB structures into the number of residues for each protein chain

In [ ]:
chains = pdb.flatMap(StructureToPolymerChains(False,True)) \
            .filter(PolymerComposition(PolymerComposition.AMINO_ACIDS_20))
chains.count()

### Build a dataset out of the PDB chain information

In [ ]:
def calcProperties(c):
    return Row(c[0], c[1].num_groups)

rows = chains.map(lambda c: calcProperties(c))

In [ ]:
col_names = ["chainId", "residues"]
summary = pythonRDDToDataset.get_dataset(rows, col_names)
summary.describe(col_names[1:]).toPandas()

In [ ]:
chains_df = summary.toPandas()
chains_df.head(5)

### Remove duplicate entries and aggregate the range covered per PDB ID by PFAM families

In [ ]:
df_noDups = df_noDups.groupby(by='chainId')[['range']].sum()
df_noDups.head(5)

### Joining the PDB data with PFAM data

In [ ]:
joined_df = chains_df.set_index('chainId').join(df_noDups, how="left")
joined_df.head(5)

In [ ]:
joined_df['range'] = joined_df[['range']].fillna(0)
joined_df.head(5)

In [ ]:
joined_df['coverage'] = joined_df['range'] / joined_df['residues']

In [ ]:
joined_df.head(10)

## Calculate the distribution of sequence fraction

In [ ]:
plot = matplotlib.pyplot.hist(joined_df['coverage'], bins=50)

## Display the structures with the lowest classification coverage

Display all the structures within some range of PFAM coverage using view_structure and passing a list of PDB IDs

In [ ]:
#low_coverage_hits = joined_df[(joined_df['coverage'] < 0.75) & (joined_df['coverage'] > 0.25)]
low_coverage_hits = joined_df[joined_df['coverage'] < 0.1]
view_structure(list(low_coverage_hits.index))

## New Pfam families from unclassified structural regions

Some of the unclassified structural regions could potentially be built into new sequence families.

For example, 3QZ0 chain A is unclassified in Pfam but classified in ECOD as a domain of unknown function with a single sequence in the family: http://prodata.swmed.edu/ecod/complete/family/EF19620. A new family could be created by searching sequence homologs 

In [ ]:
spark.stop()